In [1]:
from selenium import webdriver
import requests
import time
 
import requests
from bs4 import BeautifulSoup  # Beautiful Soup是一个可以从HTML或XML文件中提取结构化数据的Python库
    
option = webdriver.ChromeOptions()
option.add_argument("disable-infobars")
option.add_argument("--headless")
 
driver = webdriver.Chrome(chrome_options = option)
driver.get("https://www.jiqizhixin.com/dailies")
# 将网页拉倒最下方
driver.maximize_window()
time.sleep(2)
 
def execute_times(times):
    for i in range(times + 1):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
execute_times(10)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [2]:
driver.find_element_by_xpath('//*[@id="js-has-modal"]/div[2]/div/div/span').click()

In [3]:
for i in range(100):#这里设置点击100次“加载更多”
    driver.find_element_by_link_text("加载更多").click()
    time.sleep(5) #如果网页没有完全加载，会出现点击错误，所以加了一个睡眠时间。

    soup = BeautifulSoup(driver.page_source, 'html.parser')

In [4]:
import re

In [5]:
# 找到loading-wrapper
loading_wrapper_items = soup.find('div', class_=re.compile('loading-wrapper'))


In [6]:
# 找到所有的"daily-every js-daily-every"，这其中包含当前所加载出来的网页中的所有文章及日期
temp = loading_wrapper_items.find_all('div',class_="daily-every js-daily-every")


In [7]:
# all_articles 存储在所有日期下，当前日期的所有文章title和contents
all_articles = {}
each_article = {}

for item in temp:
    
    #temp = loading_wrapper_items.find_all('div',class_="daily-every js-daily-every")
    
    # 找到所有的文章，store in article_list 
    article_list = item.find_all('div',class_="daily-every__list")
    
    #print(type(item.find("div", class_="daily-every__time js-daily-time "))
    
    if (item.find("div", class_= ("daily-every__time js-daily-time ")) == None): 
        break
    else:
        # 找到当前文章的日期 daily_every_items_date
        daily_every_items_date = item.find("div", class_= ("daily-every__time js-daily-time ")).text
    #print(daily_every_items_date)
    
    # 提取当前日期下所有文章的title 存到 all_title
    all_title = []
    # 提取当前日期下所有文章的content 存到 all_contents
    all_contents = []
    # 提取当前日期下，titile及content， 存到article_day
    article_day = []
    for each_item in article_list:
        
        # 在（"a", class_="u-text-limit--two daily-every__title js-open-modal"）中找到当前日期下所有文章的title
        all_article_title = each_item.find_all("a", class_="u-text-limit--two daily-every__title js-open-modal")
        
        # 在（"div", class_="u-text-limit--three daily__content""）中找到当前文章的content
        all_article_contents = each_item.find_all("div", class_="u-text-limit--three daily__content")
        
        for each_article in each_item:
            # 在all_article_title中找到每篇文章的title，“each_article_title”
            each_article_title = each_article.find("a", class_="u-text-limit--two daily-every__title js-open-modal").text
            # 在all_article_contents中找到每篇文章的content， “each_article_contents”
            each_article_contents = each_article.find("div", class_="u-text-limit--three daily__content").text
            # each_article存储每篇文章的title及content
            each_article = {
               "title" : each_article_title,
               "contents" : each_article_contents
            }
            # 将当前的文章的title和content存到article_day list中
            article_day.append(each_article)
            
    # 将当前日期下的article_day存到all_articles [daily_every_items_date]中
    all_articles [daily_every_items_date] = article_day
    

In [8]:
target_article = {}

# 从all_articles中找出符合条件的2月20-2月27日的所有文章title和contents
for i in range(8):
    target_article.update({"2月2"+ str(i): all_articles["2月2"+ str(i)]})
#print(target_article)

In [9]:
import csv
import codecs

# 输出csv文件，表头为fileHeader
fileHeader = ["date", "title","contents"]
# 写入数据
csvFile = codecs.open("JiQiZhixin.csv","w",'utf_8_sig')
writer = csv.writer(csvFile)
writer.writerow(fileHeader)



In [10]:
for date in target_article:
    for article in target_article[date]:
        #print(article)
        title = article['title']
        contens = article['contents']
        cur_row = [date, title, contens]
        writer.writerow(cur_row)
csvFile.close()